In [1]:
import gurobipy as gu

In [2]:
import pandas as pd

C:\Users\serha\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\serha\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\serha\anaconda3\lib\site-packages\numpy\.libs\libopenblas.fb5ae2tyxyh2ijrdkgdgq3xbklktf43h.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
C:\Users\serha\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\serha\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (
C:\Users\serha\AppData\Local\Temp\ipykernel_34384\4080736814.py:1: DeprecationWarning: 
Pyarrow will become a re

In [3]:
import numpy as np

In [4]:
import time

# Initialize variables

In [5]:
filename = "example_data.csv"  # Example file name
rollWidth = 100
iteration = 0  # Track the number of iterations
start_time = time.time()  # Record start time

In [6]:
results = pd.DataFrame(columns=[
    'File', 'RollWidth', 'Status', 'CG_Iterations', 'GAP', 'OptimalSolution', 'Time'
])

# Define the InitialPatternsGenerator class


In [7]:
class InitialPatternsGenerator:
    def __init__(self, inputDf, rollWidth):
        self.inputDf = inputDf  # DataFrame containing item sizes and amounts
        self.rollWidth = rollWidth
        self.nbItems = len(inputDf)  # Number of items
        self.patternDf = None  # Placeholder for patterns DataFrame

    def generateBasicInitialPatterns(self):
        # Initialize the DataFrame to store patterns
        columns = ['PatternCost', 'PatternFill']
        patterns = pd.DataFrame(index=range(self.nbItems), columns=columns)

        # Generate initial patterns based on rollWidth / size
        patterns['PatternCost'] = 1  # Default cost for each pattern
        patterns['PatternFill'] = [
            [self.rollWidth // size if i == idx else 0 for i, size in enumerate(self.inputDf['Size'])]
            for idx in range(self.nbItems)
        ]

        self.patternDf = patterns
        return self.patternDf


# Define the MasterProblem class


In [8]:
class MasterProblem:
    def __init__(self, patternDf, inputDf):
        self.patternCost = patternDf['PatternCost'].values
        self.patternFill = patternDf['PatternFill'].tolist()  # Convert to list
        self.amount = inputDf['Amount'].values
        self.model = gu.Model("MasterProblem")
        self.patternsIndex = patternDf.index.values

    def buildModel(self):
        self.patternUseVar = self.model.addVars(
            self.patternsIndex, lb=0, vtype=gu.GRB.INTEGER, name="PatternUseVar"
        )
        self.model.addConstrs(
            (gu.quicksum(
                self.patternFill[p][j] * self.patternUseVar[p]
                for p in self.patternsIndex
            ) >= self.amount[j] for j in range(len(self.amount))),
            "DemandConstraints"
        )
        self.model.setObjective(
            gu.quicksum(self.patternCost[p] * self.patternUseVar[p] for p in self.patternsIndex),
            sense=gu.GRB.MINIMIZE
        )
        self.model.update()

    def solveRelaxedModel(self):
        self.relaxedModel = self.model.relax()
        self.relaxedModel.optimize()
        if self.relaxedModel.Status != gu.GRB.OPTIMAL:
            raise Exception("Relaxed Master Problem not solved to optimality.")

    def getDuals(self):
        if self.relaxedModel.Status == gu.GRB.OPTIMAL:
            return self.relaxedModel.getAttr("Pi", self.relaxedModel.getConstrs())
        else:
            raise Exception("Duals not available because relaxed model is not optimal.")

    def addColumn(self, objective, newPattern):
        # Generate a unique name for the new variable
        varName = f"PatternUseVar[{len(self.model.getVars())}]"
        # Create a new column using the new pattern and associated constraints
        newColumn = gu.Column(newPattern, self.model.getConstrs())
        # Add a new variable to the model
        self.model.addVar(
            vtype=gu.GRB.INTEGER, 
            lb=0, 
            obj=objective, 
            column=newColumn, 
            name=varName
        )
        self.model.update()
        
        # Append the new pattern to the patternFill list
        self.patternFill.append(newPattern)

    def solveModel(self, timeLimit=None, GAP=None):
        if timeLimit:
            self.model.setParam('TimeLimit', timeLimit)
        if GAP:
            self.model.setParam('MIPGap', GAP)
        self.model.optimize()

    def getObjectiveValue(self, rounded=True):
        return np.rint(self.model.objVal) if rounded else self.model.objVal

    def getStatus(self):
        return self.model.Status

    def getGAP(self):
        return self.model.MIPGap

# Define the Subproblem class

In [9]:
class Subproblem:
    def __init__(self, inputDf, rollWidth, duals):
        self.pieceSize = inputDf['Size'].values
        self.rollWidth = rollWidth
        self.duals = duals
        self.model = gu.Model("Subproblem")
        self.piecesIndex = inputDf.index.values  # Ensure indices match inputDf

    def buildModel(self):
        # Define variables
        self.piecesInPatternVar = self.model.addVars(
            self.piecesIndex, lb=0, vtype=gu.GRB.INTEGER, name="PiecesInPatternVar"
        )
        
        # Add roll width constraint
        self.model.addConstr(
            gu.quicksum(self.pieceSize[j] * self.piecesInPatternVar[j] for j in self.piecesIndex) <= self.rollWidth,
            "RollWidthConstraint"
        )
        
        # Set objective using duals
        # Ensure duals[j] is properly indexed to match self.piecesIndex
        self.model.setObjective(
            gu.quicksum(self.piecesInPatternVar[j] * self.duals[j] for j in self.piecesIndex),
            sense=gu.GRB.MAXIMIZE
        )

    def solveModel(self, timeLimit=None, GAP=None):
        if timeLimit:
            self.model.setParam('TimeLimit', timeLimit)
        if GAP:
            self.model.setParam('MIPGap', GAP)
        self.model.optimize()

    def getObjectiveValue(self):
        return self.model.objVal

    def getNewPattern(self):
        return self.model.getAttr("X", self.model.getVars())

# Define the input data


In [10]:
inputDf = pd.DataFrame({
    'Size': [70, 50, 25, 15, 8],
    'Amount': [6, 11, 17, 35, 21]
})
rollWidth = 100

In [11]:
inputDf

,Size,Amount
0,70,6
1,50,11
2,25,17
3,15,35
4,8,21


# Main process
# Step 1: Generate initial patterns

In [12]:
patternGenerator = InitialPatternsGenerator(inputDf, rollWidth)

In [13]:
patternDf = patternGenerator.generateBasicInitialPatterns()

In [14]:
patternDf

,PatternCost,PatternFill
0,1,"[1, 0, 0, 0, 0]"
1,1,"[0, 2, 0, 0, 0]"
2,1,"[0, 0, 4, 0, 0]"
3,1,"[0, 0, 0, 6, 0]"
4,1,"[0, 0, 0, 0, 12]"


# Step 2: Build Master Problem


In [15]:
master = MasterProblem(patternDf, inputDf)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-07-05


In [16]:
master.buildModel()

# Step 3: Iterative improvement loop


In [17]:
modelImprovable = True

In [18]:
while modelImprovable:
    print("This is iteration number: ", iteration, "\n")
    iteration += 1  # Increment iteration count
    
    # Solve relaxed Master Problem
    master.solveRelaxedModel()
    duals = master.getDuals()

    # Solve Subproblem
    subproblem = Subproblem(inputDf, rollWidth, duals)
    subproblem.buildModel()
    subproblem.solveModel()

    # Check if improvement exists
    modelImprovable = subproblem.getObjectiveValue() > 1
    if modelImprovable:
        newPatternCost = 1
        newPatternCuts = subproblem.getNewPattern()
        print("\nThe new pattern cut is: ", newPatternCuts, "\n")
        master.addColumn(newPatternCost, newPatternCuts)

This is iteration number:  0 

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 5 rows, 5 columns and 5 nonzeros
Model fingerprint: 0x7fdca912
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+00, 4e+01]
Presolve removed 5 rows and 5 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.3333333e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.333333333e+01
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 1 rows, 5 columns and 5 nonzeros
Model fingerprint: 0xbc8179fb
Variable types: 0 continuous, 5 integer (0 binary)
Co

Thread count was 1 (of 16 available processors)

Solution count 2: 1 0.95 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+00, best bound 1.000000000000e+00, gap 0.0000%


In [19]:
master.solveModel()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 5 rows, 9 columns and 13 nonzeros
Model fingerprint: 0xfe7eaa53
Variable types: 0 continuous, 9 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+00, 4e+01]
Found heuristic solution: objective 34.0000000
Presolve removed 2 rows and 4 columns
Presolve time: 0.00s
Presolved: 3 rows, 5 columns, 8 nonzeros
Variable types: 0 continuous, 5 integer (0 binary)

Root relaxation: objective 2.138000e+01, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   21.38000    0    3   34.00000   21.38000  37.1%     -    0s
H    0     0                      22.0000000   21.38000  2

# Calculate elapsed time

In [20]:
time_elapsed = time.time() - start_time

# Append results

In [21]:
new_row = pd.DataFrame([{
    'File': filename,
    'RollWidth': rollWidth,
    'Status': master.getStatus(),
    'CG_Iterations': iteration,
    'GAP': master.getGAP(),
    'OptimalSolution': np.rint(master.getObjectiveValue(False)),
    'Time': time_elapsed
}])

In [22]:
results = pd.concat([results, new_row], ignore_index=True)

C:\Users\serha\AppData\Local\Temp\ipykernel_34384\1487284922.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, new_row], ignore_index=True)


In [23]:
results

,File,RollWidth,Status,CG_Iterations,GAP,OptimalSolution,Time
0,example_data.csv,100,2,5,0.0,22.0,0.317754


In [24]:
print("\nDetailed Patterns and Their Usage:")
for var in master.model.getVars():
    if var.X > 0:  # Only display patterns that are used
        pattern_index = int(var.VarName.split('[')[1].split(']')[0])  # Extract pattern index from variable name
        pattern = master.patternFill[pattern_index]
        print(f"Pattern {pattern_index} (Usage: {var.X}): {pattern}")



Detailed Patterns and Their Usage:
Pattern 1 (Usage: 6.0): [0, 2, 0, 0, 0]
Pattern 2 (Usage: 4.0): [0, 0, 4, 0, 0]
Pattern 5 (Usage: 6.0): [1, 0, 0, 2, 0]
Pattern 7 (Usage: 5.0): [0, 0, 0, 4, 5]
Pattern 8 (Usage: 1.0): [0, 0, 1, 5, 0]


In [25]:
# Initialize an empty list to store pattern information
pattern_data = []

In [26]:
# Collect detailed patterns and their usage
# Collect detailed patterns and their usage
for var in master.model.getVars():
    #if var.X > 0:  # Only display patterns that are used
        pattern_index = int(var.VarName.split('[')[1].split(']')[0])  # Extract pattern index from variable name
        pattern = [int(value) for value in master.patternFill[pattern_index]]  # Convert pattern values to integers
        pattern_data.append({
            'Pattern Index': pattern_index,
            'Usage': int(var.X),  # Convert usage to integer
            'Pattern': pattern
        })


In [27]:
output = pd.DataFrame(pattern_data)

In [28]:
output 

,Pattern Index,Usage,Pattern
0,0,0,"[1, 0, 0, 0, 0]"
1,1,6,"[0, 2, 0, 0, 0]"
2,2,4,"[0, 0, 4, 0, 0]"
3,3,0,"[0, 0, 0, 6, 0]"
4,4,0,"[0, 0, 0, 0, 12]"
5,5,6,"[1, 0, 0, 2, 0]"
6,6,0,"[0, 0, 0, 6, 1]"
7,7,5,"[0, 0, 0, 4, 5]"
8,8,1,"[0, 0, 1, 5, 0]"
